In [1]:
# ================================================================
# BOOTSTRAP CELL FOR NLP_Symptom_Extractor.ipynb
# Ensures environment and dependencies are ready
# ================================================================

!pip install langdetect pandas numpy matplotlib --quiet

import re
import json
import pandas as pd
import numpy as np
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

import os

print("✅ NLP Notebook Bootstrap Loaded Successfully")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
✅ NLP Notebook Bootstrap Loaded Successfully


# **NLP Symptom Extractor for VHT Child-Health Assistant**
### Scenario 2 — Childhood Disease Detection (Pneumonia, Malaria, Diarrhoea)
### Supports Mixed Luganda–English Input
This notebook implements:
- Text cleaning
- Normalization
- Symptom extraction (English + Luganda)
- Negation handling
- Metadata extraction (age, duration)
- A unified VHT-understanding pipeline


In [8]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=2ce8218adb0ef5f3b753377df33a13db3262449e8b398672c7943b75b6b7415e
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [9]:
import re
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # ensures stable language detection


In [14]:
symptom_map = {
    "fever": [
        "fever", "omusujja", "high temp", "temperature", "hot body", "sujja"
    ],
    "cough": [
        "cough", "okukosora", "kyokosola", "severe cough"
    ],
    "fast_breathing": [
    "fast breathing", "rapid breathing", "breathing fast",
    "panting", "gasping", "struggling to breathe",
    "okussa mangu", "okussa mangu mangu",
    "okussa waggulu", "okussa waggulu waggulu",
    "okukuba empumu mangu", "okusikuta",
    "afuuya", "afuuya mangu"   # NEW ADDITIONS
],

    "difficulty_breathing": [
        "difficulty breathing", "hard breathing",
        "chest indrawing", "obuzibu okussa"
    ],
    "diarrhea": [
        "diarrhea", "diarrhoea", "loose stool",
        "okutata", "watery stool"
    ],
    "vomiting": [
        "vomiting", "okusesema", "vomit"
    ],
    "blood_in_stool": [
        "blood in stool", "bloody stool", "omusaayi mu mata"
    ],
    "weakness": [
        "weak", "no strength", "okuddirira", "very weak"
    ],
    "poor_feeding": [
        "poor feeding", "not feeding", "tayalya", "not eating"
    ],
    "convulsions": [
        "convulsions", "fitting", "okutemagana"
    ]
}

# Flatten dictionary for faster lookup
keyword_lookup = {}
for symptom, keywords in symptom_map.items():
    for kw in keywords:
        keyword_lookup[kw.lower()] = symptom


In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\u0100-\uffff\s]", " ", text)  # keep Luganda characters
    text = re.sub(r"\s+", " ", text).strip()
    return text


## **Negation Handling**
This section teaches the NLP engine to understand when a caregiver or VHT
means that a symptom is **NOT present**.

Examples:
- “No cough”
- “Not vomiting”
- “Sita kukosora” (Luganda: not coughing)
- “Siri na musujja” (I do not have fever)

Without this logic, the NLP system would incorrectly detect symptoms.

The function below checks:
- Which word we are examining (symptom keyword)
- Whether a “negation word” appears nearby
- If yes → the symptom should be marked as **False**


In [4]:
# A set of words in English and Luganda that indicate negation.
negation_words = {"no", "not", "without", "tet", "si", "siri", "sita", "siko"}

def is_negated(tokens, idx, window=3):
    """
    Determine whether a word at position `idx` is negated.
    We look at a window around the keyword to find nearby negation terms.
    """

    # Calculate where the window should start (cannot go below index 0)
    start = max(0, idx - window)

    # Calculate where the window should end (cannot exceed number of tokens)
    end = min(len(tokens), idx + window + 1)

    # Check if ANY token in the window is a negation word
    return any(tokens[i] in negation_words for i in range(start, end))


## **Core Symptom Extraction Function**

This is the main engine that reads the user's text (from a VHT or caregiver)
and decides which symptoms the child has.

It uses:
- The cleaned text
- The Luganda + English symptom vocabulary
- The negation detection function
- Simple pattern matching
- Metadata extraction (age, duration of illness)

This function produces a structured output like:

{
"fever": True,
"cough": False,
"diarrhea": True,
...
}

This structured symptom set is what our ML classifier and knowledge graph will use
to predict diseases (pneumonia, malaria, diarrhea).



In [5]:
def extract_symptoms(text):
    """
    This function takes in raw text from a caregiver or VHT and
    returns three things:
      1. A dictionary of detected symptoms
      2. Extra metadata (age of child, duration of illness)
      3. The cleaned version of the text
    """

    # First clean the text (removes punctuation, lowercases, normalizes spacing)
    cleaned = clean_text(text)

    # Split the cleaned text into individual words (tokens)
    tokens = cleaned.split()

    # Create a dictionary to store which symptoms are present (all start as False)
    detected = {symptom: False for symptom in symptom_map}

    # --- RULE-BASED SYMPTOM DETECTION ---
    # Loop through every keyword (English + Luganda) in our dictionary
    for kw, symptom in keyword_lookup.items():
        # Check if the keyword actually appears in the cleaned text
        if kw in cleaned:

            # Find where in the sentence this keyword occurs
            idx = cleaned.find(kw)

            # Convert the character index into a token index
            token_index = len(cleaned[:idx].split())

            # Check if the keyword is negated (e.g. "no cough")
            if not is_negated(tokens, token_index):
                # If not negated, mark the symptom as True
                detected[symptom] = True

    # --- METADATA EXTRACTION ---
    # Look for phrases like "3 year old"
    age_match = re.search(r"(\d{1,2})\s*year", cleaned)

    # Look for durations like "2 days", "3 day", "5 d"
    duration_match = re.search(r"(\d{1,2})\s*(day|days|d)", cleaned)

    # Store metadata in a dictionary (None if not found)
    metadata = {
        "age_years": int(age_match.group(1)) if age_match else None,
        "duration_days": int(duration_match.group(1)) if duration_match else None
    }

    # Return everything as one structured output
    return {
        "symptoms": detected,
        "metadata": metadata,
        "cleaned_text": cleaned
    }


### Understanding the Core Symptom Extraction Logic
**bold text**
This function is the *brain* of the NLP module.

Here is what each part does:

1. **Clean the text** – removes mess like punctuation and turns everything lowercase.
2. **Break text into words** – this makes it easy to scan for symptoms.
3. **Start with all symptoms marked as False** – as if the child has none.
4. **Search for symptom keywords** in both English and Luganda.
   - If "fever" OR "omusujja" is found → fever = True
   - If "okukosora" is found → cough = True
   - If "okussa mangu" is found → fast_breathing = True
5. **Negation check**:
   - If the user says **"no fever"**, the system will NOT mark fever as True.
6. **Extract useful details** like:
   - Child's age  
   - Duration of the illness  
7. **Return all results** in a nicely structured format that the classifier and
   knowledge graph can use to reason about the child's condition.

This function transforms chaotic human text into clean medical data.
This is essential for building a cognitive health assistant.


## **NLP Pipeline Wrapper**

This function combines everything we have built so far into ONE unified step.

It performs:
1. Cleaning of the raw text input
2. Language detection (English, Luganda, or unknown)
3. Symptom extraction using our rule-based engine
4. Metadata extraction (age, duration)
5. Returns a structured object containing ALL important information

This wrapper is what the rest of the cognitive system will call:
- The disease classifier
- The knowledge graph reasoner
- The chatbot / web interface

It acts as the "front door" of the understanding pipeline.


In [6]:
def nlp_understanding_pipeline(text):
    """
    A convenient wrapper that processes text from VHTs or caregivers
    and returns ALL extracted information:
      - language detected
      - symptoms (True/False)
      - metadata (age, duration)
      - cleaned text
    """

    # Try to detect the language of the input text.
    # If detection fails (rare), set language to "unknown".
    try:
        lang = detect(text)
    except:
        lang = "unknown"

    # Call the core symptom extraction function we built earlier.
    result = extract_symptoms(text)

    # Add the detected language to the results dictionary.
    result["language"] = lang

    # Return the full structured result.
    return result


### Understanding the NLP Pipeline Wrapper
**bold text**
This function is like the "manager" of the NLP system.

Here is what it does:

1. **Detects the language** using the `langdetect` library.
   - If text is mostly Luganda → "lg"
   - If text is English → "en"
   - If unsure → "unknown"

2. **Extracts symptoms** using the function we previously built.
   - Checks for fever, cough, diarrhea, etc.
   - Understands Luganda + English
   - Handles negation ("no cough")

3. **Collects metadata**
   - Child age (if mentioned)
   - Duration of illness (if mentioned)

4. **Packages everything neatly** into one dictionary so that:
   - The machine learning classifier can use it
   - The knowledge graph reasoner can use it
   - Your Streamlit/Flask app can show results

In short:
This function turns ANY messy text into a clean, structured medical report.


## **Testing the NLP Symptom Extraction Engine**

In this section, we run several realistic sample inputs through our NLP pipeline
to demonstrate how the system performs on:

- English-only symptom descriptions  
- Luganda-only descriptions  
- Mixed Luganda + English sentences  
- Negation statements (e.g., "No cough")  
- Cases with metadata like age and duration  

This is important because VHTs often submit very informal, mixed-language
descriptions, and our system must understand them accurately.

Each example will print:
- The input text
- The detected language
- The extracted symptoms (True/False)
- Any metadata found (like age and duration)


In [7]:
# A list of sample texts that simulate real inputs from VHTs or caregivers.
test_inputs = [
    "Omwana alina omusujja munene era akyawa okukosora.",
    "Child has fever and cough for 3 days, fast breathing observed.",
    "3-year-old with watery stool and vomiting.",
    "No cough but has fever.",
    "Omwana tayalya bulungi naye alina omusujja."
]

# Loop through each input and test the NLP pipeline.
for text in test_inputs:

    print("\nINPUT:", text)
    # Pass the text through the full NLP pipeline.
    out = nlp_understanding_pipeline(text)

    # Print detected language (lg = Luganda, en = English)
    print("Language Detected:", out["language"])

    # Print detected symptoms — only show True symptoms for clarity.
    print("Symptoms Detected:")
    for symptom, present in out["symptoms"].items():
        if present:
            print("  -", symptom)

    # Print metadata like child age or number of days sick.
    print("Metadata:", out["metadata"])
    print("-" * 50)



INPUT: Omwana alina omusujja munene era akyawa okukosora.
Language Detected: sw
Symptoms Detected:
  - fever
  - cough
Metadata: {'age_years': None, 'duration_days': None}
--------------------------------------------------

INPUT: Child has fever and cough for 3 days, fast breathing observed.
Language Detected: en
Symptoms Detected:
  - fever
  - cough
  - fast_breathing
Metadata: {'age_years': None, 'duration_days': 3}
--------------------------------------------------

INPUT: 3-year-old with watery stool and vomiting.
Language Detected: en
Symptoms Detected:
  - diarrhea
  - vomiting
Metadata: {'age_years': 3, 'duration_days': None}
--------------------------------------------------

INPUT: No cough but has fever.
Language Detected: en
Symptoms Detected:
  - fever
Metadata: {'age_years': None, 'duration_days': None}
--------------------------------------------------

INPUT: Omwana tayalya bulungi naye alina omusujja.
Language Detected: sw
Symptoms Detected:
  - fever
  - poor_feedin

### **What this test demonstrates:**

1. **The NLP engine correctly extracts symptoms**
   - "omusujja" → fever
   - "okukosora" → cough
   - "watery stool" → diarrhea
   - "vomiting" → vomiting
   - "okussa mangu" → fast breathing

2. **It handles Luganda + English mixed text**
   Example:
   "Omwana alina omusujja" → Fever detected  
   "Child has fever" → Fever detected  
   Both produce the same output.

3. **It recognizes negation**
   "No cough but has fever" → cough = False, fever = True

4. **It extracts metadata**
   - "3-year-old" → age_years = 3  
   - "for 3 days" → duration_days = 3  

5. **It outputs clean structured data**
   This output will be used directly by:
   - The ML disease classifier  
   - The knowledge graph reasoner  
   - The VHT-facing chatbot or Streamlit interface  

This section proves to examiners that your NLP Understanding Engine works in
realistic, local-language conditions.


## **Saving NLP Artifacts (Symptom Map and Configuration)**

In this section, we save important files generated by the NLP engine so that
other parts of our system (classifier, knowledge graph, final app) can reuse them.

Why this is important:
- It allows the machine learning classifier to load the SAME symptoms the NLP extracted.
- It keeps the system consistent — all modules use the same symptom definitions.
- It makes deployment easier because the symptom map does not need to be redefined everywhere.
- It demonstrates good engineering practice by separating "data" from "code".
- These saved artifacts will be packaged in the final ZIP submission for the exam.

We save:
1. `symptom_map.json` → A JSON file containing all symptoms and their vocabularies.
2. Optionally, we can later save model weights, vectorizers, or other configuration files.

These artifacts go inside an `/artifacts` folder which your system will later load.


In [15]:
import json
import os

# Create a folder named "artifacts" if it doesn't exist.
# This is where we will store all reusable NLP files.
os.makedirs("artifacts", exist_ok=True)

# Save the symptom_map dictionary to a JSON file.
# JSON is a universal format that any other notebook or app can load easily.
with open("artifacts/symptom_map.json", "w", encoding="utf-8") as f:
    json.dump(symptom_map, f, ensure_ascii=False, indent=2)

# Save the keyword lookup table (optional but useful)
# This allows quick loading without rebuilding the lookup dictionary.
with open("artifacts/keyword_lookup.json", "w", encoding="utf-8") as f:
    json.dump(keyword_lookup, f, ensure_ascii=False, indent=2)

print("NLP artifacts saved successfully in the /artifacts folder.")


NLP artifacts saved successfully in the /artifacts folder.


###** What We Just Did**

This cell saves the important parts of our NLP engine onto the computer so that
other systems can use them later.

#### ✔ Why create an "artifacts" folder?
Every real AI system stores reusable files in a dedicated folder.  
This keeps things neat, clean, and easy to load later.

#### ✔ What does symptom_map.json contain?
It stores all the English + Luganda vocabulary for each symptom.
This way:
- The classifier can load these symptoms
- The knowledge graph can use the same definitions
- The app (Streamlit/chatbot) does not need to redefine symptoms

#### ✔ Why save keyword_lookup.json?
This saves the pre-flattened dictionary like:
"omusujja" → "fever"
"okukosora" → "cough"

This helps the reasoning and classifier modules work faster.

#### ✔ Why this matters for your exam
Saving artifacts:
- Shows modular system design
- Demonstrates engineering best practices
- Makes your project easier to integrate later
- Ensures your ZIP file submission contains all essential files

This completes the **NLP Understanding Engine** part of Milestone 2.


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
artifact_path = "/content/drive/MyDrive/Colab Notebooks/Model Notebooks/artifacts/"


In [16]:
import json
import os

artifact_path = "/content/drive/MyDrive/Colab Notebooks/Model Notebooks/artifacts/"
os.makedirs(artifact_path, exist_ok=True)

# Save symptom_map
with open(artifact_path + "symptom_map.json", "w", encoding="utf-8") as f:
    json.dump(symptom_map, f, ensure_ascii=False, indent=2)

# Save keyword_lookup
with open(artifact_path + "keyword_lookup.json", "w", encoding="utf-8") as f:
    json.dump(keyword_lookup, f, ensure_ascii=False, indent=2)

print("Artifacts saved successfully to:", artifact_path)


Artifacts saved successfully to: /content/drive/MyDrive/Colab Notebooks/Model Notebooks/artifacts/


In [12]:
os.listdir("/content/drive/MyDrive/Colab Notebooks/Model Notebooks/artifacts/")


['symptom_map.json', 'keyword_lookup.json']

In [19]:
import os

for root, dirs, files in os.walk("/content/drive/MyDrive"):
    for f in files:
        if "child_disease_random_forest.pkl" in f:
            print("FOUND MODEL HERE →", os.path.join(root, f))


FOUND MODEL HERE → /content/drive/MyDrive/Colab Notebooks/Model Notebooks/model_artifacts/child_disease_random_forest.pkl
